<a href="https://colab.research.google.com/github/ReiAkio/AI_Project_Semester2/blob/main/Maua_AI_Projeto_Semestre2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resumo do projeto

O projeto apresentado constitui uma síntese da etapa de filtragem, treinamento e testes de áudios para desenvolver modelos de Aprendizado de Máquina no contexto de aprendizado supervisionado, conforme proposto no Trabalho de Conclusão de Curso (TCC) intitulado "Reconhecimento de Voz por Inteligência Artificial em um Ambiente Virtual". O objetivo central é identificar, a partir de arquivos de áudio, o locutor específico que está falando. Para tal, os dados passam por um processo de filtragem por meio da Transformada de Fourier, seguido por uma etapa de preparação para garantir a organização apropriada. Posteriormente, é realizada a normalização dos dados, assegurando uma comparação justa entre eles. Em seguida, diversos classificadores são aplicados, como Regressão Logística, Máquinas de Vetores de Suporte (SVM), K-Vizinhos Mais Próximos (KNN) e Redes Neurais. O relatório de classificação apresentado proporciona uma análise comparativa da precisão das previsões nos dados de áudio, para a identificação do locutor.

## Nome dos integrantes



*   Cindy Natsuki Yoshita RA: 19.00633-0
*   Gabriel Belapetravicius Dias RA: 18.00487-3
*   Raphael Gueleri Kalaes RA: 18.02011-9
*   Thiago Akio Kanada Tanaka RA: 19.01726-0





## Filtro dos dados

### Import das Bibliotecas

Para que seja possível tanto o filtro de voz, quanto o treinamento dos modelos de classificadores, é necessário importar as bibliotecas necessárias

In [27]:
import numpy as np
import librosa
import os
import librosa.display

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture
from sklearn.neural_network import MLPClassifier

import re

### Utlização dos dados

Essas duas linhas de código estão usando comandos de terminal para instalar o Git e clonar um repositório do GitHub diretamente no notebook do Google Colab, assim, permitindo que seja utilizado os dados armazenados nele neste notebook.

In [28]:
!apt-get install git
!git clone https://github.com/ReiAkio/AI_Project_Semester2.git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
fatal: destination path 'AI_Project_Semester2' already exists and is not an empty directory.


### Carregar arquivos de audio do path especificado e aplicar STFT

In [29]:
%pip install python_speech_features

Aqui, carrega-se um arquivo de áudio, converte o sinal de áudio para mono e calcula a Transformada de Fourier de Tempo Curto (STFT) desse sinal. Ela retorna a representação STFT.

In [30]:
def process_audio_file(file_path):
    # Carregar o arquivo de áudio
    y, sr = librosa.load(file_path, mono=True)
    # Calcular o STFT
    stft = np.abs(librosa.stft(y))
    return stft

### Extraindo arquivos de audio com STFT

Para manipular melhor as ondas sonoras, extrai-se uma variedade de características de um sinal de áudio representado pela Transformada de Fourier de Tempo Curto (STFT). Essas características incluem informações como os coeficientes cepstrais de frequência Mel (MFCCs), o espectrograma de Mel, o cromagrama, o contraste espectral, o Rolloff espectral e outras informações relevantes sobre o sinal de áudio. Essas características são essenciais para análises posteriores e podem será utilizado para treinar modelos de aprendizado de máquina na tarefaa de processamento de áudio.

In [31]:
# Função para extrair características do STFT (frequência dominante e outras)
def extract_features(stft, y, sr):
    # Calcular frequências para cada bin da FFT
    freqs = librosa.fft_frequencies(sr=22050, n_fft=2048)
    # Calcular a média do STFT em cada frequência
    mean_stft = np.mean(stft, axis=1)
    # Encontrar a frequência dominante
    dominant_freq = freqs[np.argmax(mean_stft)]

    # Calcular MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Calcular o espectrograma de Mel
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)

    # Calcular o cromagrama
    chromagram = librosa.feature.chroma_stft(S=stft, sr=sr)

    # Calcular o contraste espectral
    spectral_contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)

    # Calcular o Rolloff espectral
    spectral_rolloff = librosa.feature.spectral_rolloff(S=stft, sr=sr)

    # Calcular o Tom médio
    pitch = librosa.pitch_tuning(y)
    mean_pitch = np.mean(pitch)

    # Calcular o Ponto culminante do Spectrograma
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

    # Bandas de frequência (em Hz) - você pode ajustar esses valores conforme necessário
    low_band = (0, 1000)  # Exemplo: 0-1000 Hz
    mid_band = (1000, 4000)  # Exemplo: 1000-4000 Hz
    high_band = (4000, 20000)  # Exemplo: 4000-20000 Hz

    # Calcular a média das amplitudes nas bandas de frequência
    low_band_mean = np.mean(np.sum(mel_spectrogram[(dominant_freq >= low_band[0]) & (dominant_freq <= low_band[1])], axis=0))
    mid_band_mean = np.mean(np.sum(mel_spectrogram[(dominant_freq >= mid_band[0]) & (dominant_freq <= mid_band[1])], axis=0))
    high_band_mean = np.mean(np.sum(mel_spectrogram[(dominant_freq >= high_band[0]) & (dominant_freq <= high_band[1])], axis=0))

    # Filtrar o sinal harmônico e percussivo
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Calcular o STFT para o sinal harmônico
    stft_harmonic = librosa.stft(y_harmonic)

    # Calcular o STFT para o sinal percussivo
    stft_percussive = librosa.stft(y_percussive)

    # Calcular a energia harmônica
    harmonic_energy = np.sum(np.abs(stft_harmonic), axis=0)

    # Calcular a relação sinal-ruído harmônico
    harmonic_to_noise = np.sum(np.abs(stft_harmonic), axis=0) / np.sum(np.abs(stft_percussive), axis=0)

    # Concatenar todas as características extraídas
    features = np.concatenate((np.mean(mfccs, axis=1),
                               np.mean(mel_spectrogram, axis=1),
                               np.mean(chromagram, axis=1),
                               np.mean(spectral_contrast, axis=1),
                               [dominant_freq],
                               [np.mean(spectral_rolloff)],
                               [mean_pitch],
                               [np.max(spec_centroid)]))

    return features

## Aprendizado Supervisionado

### Rotulando por caminho

A função get_label_from_path remove o diretório e a extensão do arquivo para obter o nome básico do arquivo e, em seguida, usa expressões regulares para remover quaisquer dígitos numéricos presentes no nome do arquivo.

In [32]:
def get_label_from_path(file_path):
    # Extracts the label 'speaker' from 'speakerX.wav'
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    # Use regular expression to remove trailing digits
    label = re.sub(r'\d+', '', base_name)
    return label

### Organizando os dados para treinos e testes

Após rotular, precisamos processar uma lista de arquivos de áudio e os preparar para treinamento. Para cada arquivo de áudio na lista, carrega os dados de áudio, extrai o STFT e, em seguida, extrai características específicas desse STFT. Os recursos extraídos e os rótulos correspondentes são armazenados em listas separadas. Essas listas são então convertidas em arrays numpy. E então, a função imprime a forma dos arrays para verificação e os retorna.

In [33]:
def prepare_data(audio_files):
    X = []
    y = []
    for file_path, label in audio_files:

        y_data, sr = librosa.load(file_path)
        stft = process_audio_file(file_path)

        features = extract_features(stft,y_data, sr)


        X.append(features)
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
    return X, y

Assim, busca-se todos os arquivos de áudio no diretorio onde estão os arquivos, verificando os formatos de arquivo aceitos (.wav e .mp3). Para cada arquivo encontrado, extrai o rótulo do arquivo utilizando a função get_label_from_path. Em seguida, armazena o caminho do arquivo e o rótulo correspondente em uma lista. Ao final, imprime o número total de arquivos de áudio encontrados e chama a função prepare_data para processar esses arquivos de áudio.

In [34]:
def load_data_from_directory(directory_path):
    audio_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.wav') or file.endswith('.mp3'):
                file_path = os.path.join(root, file)
                label = get_label_from_path(file_path)
                audio_files.append((file_path, label))

    print(f"Number of audio files: {len(audio_files)}")
    return prepare_data(audio_files)

### Normalização

Precisa-se padronizar a escala dos dados, tornando-os comparáveis e facilitando o treinamento de modelos de machine learning.

In [35]:
def normalization(X_train,X_test):
    global scaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

### Preparando os dados

A cedula abaixo define os diretórios de treinamento e teste para os dados de áudio, chamam a função load_data_from_directory para carregar os dados de áudio desses diretórios e, em seguida, normalizam os conjuntos de dados de treinamento e teste com a função normalization. O processo de normalização é essencial para padronizar a escala dos dados e melhorar a eficácia dos algoritmos de aprendizado de máquina.

In [36]:
train_data_directory = 'AI_Project_Semester2/data/train_audio/AudiosDeTreino'
test_data_directory = 'AI_Project_Semester2/data/test_audio/_audios_teste'
X_train, y_train = load_data_from_directory(train_data_directory)
X_test, y_test = load_data_from_directory(test_data_directory)
normalization(X_train, X_test)


Number of audio files: 73
Shape of X: (73, 164), Shape of y: (73,)
Number of audio files: 25
Shape of X: (25, 164), Shape of y: (25,)


### Aplicando Logistic Regression

É feito o treinamento e testes utilizando o modelo Logistic Regression. A Regressão Logística é um modelo estatístico usado para prever a probabilidade de ocorrência de um evento, com base em uma série de variáveis explanatórias. É comumente utilizado para problemas de classificação binária, nos quais o objetivo é prever se uma instância pertence a uma classe específica ou não. Este modelo estima as chances de uma resposta categórica, utilizando a função logística para modelar a relação entre as variáveis independentes e a variável dependente. A regressão logística é frequentemente utilizada para interpretar a influência das variáveis independentes na probabilidade de ocorrência de um evento.

In [52]:
logreg_model = LogisticRegression(max_iter=6000)
logreg_model.fit(X_train, y_train)
logreg_predictions = logreg_model.predict(X_test)
print('Logistic Regression Accuracy: ', accuracy_score(y_test, logreg_predictions))
print('Classification Report for logistic Regression Model:')
print(classification_report(y_test, logreg_predictions))

Logistic Regression Accuracy:  0.72
Classification Report for logistic Regression Model:
              precision    recall  f1-score   support

       Acoba       1.00      1.00      1.00         1
          Ba       0.50      1.00      0.67         1
       Cindy       0.00      0.00      0.00         1
     Eduardo       1.00      1.00      1.00         2
          Ha       1.00      1.00      1.00         2
       Harry       1.00      0.50      0.67         2
       Nakai       1.00      1.00      1.00         2
   Penterist       0.33      1.00      0.50         2
     Raphael       1.00      1.00      1.00         2
       Ruivo       1.00      1.00      1.00         2
        Teko       0.00      0.00      0.00         2
         bia       0.50      1.00      0.67         1
    fernanda       0.00      0.00      0.00         1
irmamichelli       0.00      0.00      0.00         1
    michelli       0.50      1.00      0.67         1
      nicole       0.00      0.00      0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Aplicando SVM

É feito o treinamento e testes utilizando o modelo SVM. SVM (Support Vector Machine) é um algoritmo de aprendizado de máquina supervisionado usado para tarefas de classificação e regressão. Na classificação, o SVM mapeia os dados em um espaço de alta dimensionalidade para encontrar um hiperplano que melhor separa as classes de dados. Este algoritmo procura encontrar a melhor margem possível entre os pontos de dados das diferentes classes, maximizando a distância entre os pontos mais próximos de cada classe e o hiperplano. SVMs também podem lidar com conjuntos de dados não lineares, usando truques de kernel para mapear os dados em um espaço de dimensão superior. Este método é eficaz em conjuntos de dados de pequeno e médio porte e é amplamente utilizado em problemas de classificação, como reconhecimento de padrões e análise de texto.

In [38]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
print('SVM Accuracy: ', accuracy_score(y_test, svm_predictions))
print('Classification Report for SVM Model:')
print(classification_report(y_test, svm_predictions))

print('Confusion Matrix for SVM Model:')
print(confusion_matrix(y_test, svm_predictions))

SVM Accuracy:  0.44
Classification Report for SVM Model:
              precision    recall  f1-score   support

       Acoba       0.00      0.00      0.00         1
          Ba       0.50      1.00      0.67         1
       Cindy       0.00      0.00      0.00         1
     Eduardo       0.00      0.00      0.00         2
          Ha       0.00      0.00      0.00         2
       Harry       0.33      0.50      0.40         2
       Nakai       0.00      0.00      0.00         2
   Penterist       0.67      1.00      0.80         2
     Raphael       0.40      1.00      0.57         2
       Ruivo       1.00      1.00      1.00         2
        Teko       0.25      0.50      0.33         2
         bia       0.00      0.00      0.00         1
    fernanda       0.00      0.00      0.00         1
irmamichelli       0.00      0.00      0.00         1
    michelli       0.50      1.00      0.67         1
      nicole       0.00      0.00      0.00         1
      sergio       0.33 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Aplicando KNN

É feito o treinamento e testes utilizando o modelo KNN. KNN (K-Nearest Neighbors) é um algoritmo de aprendizado de máquina supervisionado que pode ser usado para classificação e regressão. O algoritmo funciona encontrando os K pontos de dados mais próximos a uma nova observação e prevendo o rótulo ou valor com base nos rótulos ou valores desses pontos vizinhos. Na classificação, o KNN atribui a classe mais comum entre os vizinhos mais próximos como o rótulo de classificação para o novo ponto de dados, enquanto na regressão, calcula a média dos valores dos vizinhos mais próximos para prever o valor. O valor de K, ou o número de vizinhos considerados, é um hiperparâmetro crítico que influencia o desempenho do modelo. O KNN é simples e fácil de implementar, sendo particularmente útil em conjuntos de dados pequenos e médios.

In [39]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)
print('KNN Accuracy: ', accuracy_score(y_test, knn_predictions))
print('Classification Report for KNN Model:')
print(classification_report(y_test, knn_predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNN Accuracy:  0.4
Classification Report for KNN Model:
              precision    recall  f1-score   support

       Acoba       0.50      1.00      0.67         1
        Akio       0.00      0.00      0.00         0
      Alexia       0.00      0.00      0.00         0
          Ba       0.33      1.00      0.50         1
       Cindy       0.00      0.00      0.00         1
     Eduardo       0.50      1.00      0.67         2
          Ha       0.00      0.00      0.00         2
       Harry       0.33      0.50      0.40         2
       Nakai       0.00      0.00      0.00         2
   Penterist       1.00      1.00      1.00         2
     Raphael       0.00      0.00      0.00         2
       Ruivo       1.00      1.00      1.00         2
        Teko       0.00      0.00      0.00         2
         bia       0.00      0.00      0.00         1
    fernanda       0.00      0.00      0.00         1
irmamichelli       0.00      0.00      0.00         1
    michelli       1.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

### Aplicando Neural Network

In [44]:
nn_clf = MLPClassifier()
nn_clf.fit(X_train, y_train)
nn_predictions = nn_clf.predict(X_test)
print('Neural Network Accuracy: ', accuracy_score(y_test, nn_predictions))
print('Classification Report for Neural Network Model:')
print(classification_report(y_test, nn_predictions))

Neural Network Accuracy:  0.48
Classification Report for Neural Network Model:
              precision    recall  f1-score   support

       Acoba       1.00      1.00      1.00         1
      Alexia       0.00      0.00      0.00         0
          Ba       1.00      1.00      1.00         1
       Cindy       0.00      0.00      0.00         1
     Eduardo       0.00      0.00      0.00         2
          Ha       0.00      0.00      0.00         2
       Harry       1.00      0.50      0.67         2
       Nakai       1.00      1.00      1.00         2
   Penterist       0.00      0.00      0.00         2
     Raphael       0.67      1.00      0.80         2
       Ruivo       1.00      1.00      1.00         2
        Teko       0.00      0.00      0.00         2
         bia       0.25      1.00      0.40         1
     eduardo       0.00      0.00      0.00         0
    fernanda       0.00      0.00      0.00         1
irmamichelli       0.00      0.00      0.00         1
  

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being se